# ***MAIN FUNCTIONALITY***

In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns

from meteo.Site import Site
from solar.SolarPVPanel import SolarPVPanel
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging

configure_logging()
pd.options.display.float_format = '{:.3f}'.format

# Set up site location and get TMY data
name = ""
address = "York St, Belfast, BT15 1ED"
client = "Ulster University"

site = Site(name=name, 
            address=address, 
            client=client, 
            latitude=54.60452, 
            longitude=-5.92860, 
            size=100)

# Set up PV panel being used
panel_kwp = 0.3538
size_m2 = 1.990
eff = 0.2237
cell_temp_coeff = -0.004
cell_NOCT = 48
lifespan = 25
pv_eol_derating = 0.88
refraction_index = 0.05

pv_panel = SolarPVPanel(panel_kwp, size_m2, eff, cell_temp_coeff, 
                        cell_NOCT, lifespan, pv_eol_derating, refraction_index)


# Set up PV arrays
num_pv_panels = 3
surface_pitch = 35
surface_azimuth = -90
albedo = 0.2
cost_per_kWp = 1250

pv_array_1 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, surface_azimuth, albedo, cost_per_kWp)
pv_array_2 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, -45)
pv_array_3 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 0)
pv_array_4 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 45)
pv_array_5 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 90)

pv_arrays = [pv_array_1, pv_array_2, pv_array_3, pv_array_4, pv_array_5]


# Run the model
pv_model = SolarPVModel(site, pv_arrays)

print("")
print(f"{sum(array.pv_kwp for array in pv_model.arrays)}kWp Solar PV modelled covering a {sum(array.area_m2 for array in pv_model.arrays)}m2 area.")


In [ ]:
pv_model.combined_model.Total_PV_Losses_kWh_Total.sum()

In [ ]:
round(pv_model.combined_model.Total_PV_Losses_kWh_Total.sum() / pv_model.combined_model.PV_Gen_kWh_Total.sum() * 100, 2)

In [ ]:
a = pv_model.combined_model.PV_Thermal_Loss_kWh_Total.sum() 
b = pv_model.combined_model.Low_Light_Loss_kWh_Total.sum() 
c = pv_model.combined_model.IAM_Loss_kWh_Total.sum()
d = a + b + c
e = pv_model.combined_model.PV_Gen_kWh_Total.sum()
f = d / e * 100

print(f"{a:.2f} kWh Thermal Losses")
print(f"{b:.2f} kWh Low Light Losses")
print(f"{c:.2f} kWh IAM Losses")
print(f"{d:.2f} kWh Total Losses")
print()
print(f"{e:.2f} kWh PV gen annual")
print(f"{f:.2f}% losses")

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.hourly.sum()

In [ ]:
print(pv_model.models[0]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[1]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[2]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[3]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[4]["model_result"].E_POA_kWm2.sum())

In [ ]:
px.bar(pv_model.summary_grouped.quarterly[["Total_PV_Losses_kWh_Total", "PV_Gen_kWh_Total"]])

# ***Performance Test***

In [22]:
# Performance Testing
# import pstats
# import cProfile
# from meteo.Site import Site
# from solar.SolarPVPanel import SolarPVPanel
# from solar.SolarPVArray import SolarPVArray
# from solar.SolarPVModel import SolarPVModel
# from misc.log_config import configure_logging
# configure_logging()
# cProfile.run("SolarPVModel(site=Site(latitude=54.60452, longitude=-5.92860, size=100), arrays=[SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4)])", 'performance_test.prof')
# p = pstats.Stats('performance_test.prof')
# p.sort_stats('tottime').print_stats()
# p.sort_stats('cumtime').print_stats()

### **Example: PV_Only model for multiple site locations and arrays:**

In [10]:
from ww_models.PVOnly import Site, SolarPVPanel, SolarPVArray, PV_Only
from misc.log_config import configure_logging
configure_logging()

# Initialise sites to be modelled
site_1 = Site(name="Belfast")
site_2 = Site(name="Paris", latitude=48.8562, longitude=2.3477)
site_3 = Site(name="Madrid", latitude=40.4202, longitude=-3.6988)
site_4 = Site(name="Cairo", latitude=30.1098, longitude=-3.6988)
site_5 = Site(name="Abuja", latitude=9.0591, longitude=31.2235)

sites = [site_1, site_2, site_3, site_4, site_5]

# Configure Solar Panels to be used in arrays
pv_panel = SolarPVPanel()
pv_panel_2 = SolarPVPanel()

# Configure Solar Arrays from panels and system characteristics
pv_array_1 = SolarPVArray(pv_panel, 3, 35, -90)
pv_array_2 = SolarPVArray(pv_panel, 3, 35, -45)
pv_array_3 = SolarPVArray(pv_panel, 3, 35, 0)
pv_array_4 = SolarPVArray(pv_panel, 3, 35, 45)
pv_array_5 = SolarPVArray(pv_panel, 3, 35, 90)

pv_array_6 = SolarPVArray(pv_panel_2, 3, 35, 90)
pv_array_7 = SolarPVArray(pv_panel_2, 3, 35, 0)
pv_array_8 = SolarPVArray(pv_panel_2, 3, 35, -90)

# Combine solar arrays into individual systems to be modelled
pv_arrays_1 = [pv_array_1, pv_array_2, pv_array_3, pv_array_4, pv_array_5]
pv_arrays_2 = [pv_array_6, pv_array_7, pv_array_8]

# Add to Project class for further work.
project = PV_Only(sites, [pv_arrays_1, pv_arrays_2])

print()
print(f"{project.pv_models.Belfast['Belfast_1'].summary.PV_Gen_kWh_Annual:.3f} kWh annually - Belfast")
print(f"{project.pv_models.Paris['Paris_1'].summary.PV_Gen_kWh_Annual:.3f} kWh annually - Paris")
print(f"{project.pv_models.Madrid['Madrid_1'].summary.PV_Gen_kWh_Annual:.3f} kWh annually - Madrid")
print(f"{project.pv_models.Cairo['Cairo_1'].summary.PV_Gen_kWh_Annual:.3f} kWh annually - Cairo")
print(f"{project.pv_models.Abuja['Abuja_1'].summary.PV_Gen_kWh_Annual:.3f} kWh annually - Abuja")


2024-03-07 23:52:23,777 - INFO - Fetching TMY data for Belfast at latitude: 54.60452, longitude: -5.9286
2024-03-07 23:52:23,778 - INFO - TMY data obtained for Belfast at latitude: 54.60452, longitude: -5.9286
2024-03-07 23:52:23,779 - INFO - *******************
2024-03-07 23:52:23,780 - INFO - Fetching TMY data for Paris at latitude: 48.8562, longitude: 2.3477
2024-03-07 23:52:23,782 - INFO - TMY data obtained for Paris at latitude: 48.8562, longitude: 2.3477
2024-03-07 23:52:23,782 - INFO - *******************
2024-03-07 23:52:23,783 - INFO - Fetching TMY data for Madrid at latitude: 40.4202, longitude: -3.6988
2024-03-07 23:52:23,783 - INFO - TMY data obtained for Madrid at latitude: 40.4202, longitude: -3.6988
2024-03-07 23:52:23,784 - INFO - *******************
2024-03-07 23:52:23,784 - INFO - Fetching TMY data for Cairo at latitude: 30.1098, longitude: -3.6988
2024-03-07 23:52:23,785 - INFO - TMY data obtained for Cairo at latitude: 30.1098, longitude: -3.6988
2024-03-07 23:52:23


4741.066 kWh annually - Belfast
6158.238 kWh annually - Paris
8960.507 kWh annually - Madrid
10541.141 kWh annually - Cairo
8977.115 kWh annually - Abuja


In [ ]:
for site in project.site:
    print(site.tmy_data["G(h)"].sum()/1000)

943.7153500000001
1233.6499
1826.7363500000001
2271.17575
2109.4402


In [24]:
for site in project.pv_models.keys():
    print(f"Site Name: {site}")
    model_indices = project.pv_models[site].keys()
    print(f"Site Models: {list(model_indices)}")

Site Name: Belfast
Site Models: ['Belfast_1', 'Belfast_2']
Site Name: Paris
Site Models: ['Paris_1', 'Paris_2']
Site Name: Madrid
Site Models: ['Madrid_1', 'Madrid_2']
Site Name: Cairo
Site Models: ['Cairo_1', 'Cairo_2']
Site Name: Abuja
Site Models: ['Abuja_1', 'Abuja_2']


In [ ]:
project.pv_models.Belfast_1.summary_grouped.daily

In [ ]:
project.pv_models.Belfast_1.summary

In [ ]:
project.pv_models.Abuja_1.summary

# ***Example Running Model with variable inputs***

In [ ]:
# Most direct implementation using mainly default values, setting coordinates, and allocating 3 panels @ 35 degrees pitch
from solar.SolarPVModel import Site, SolarPVPanel, SolarPVArray, SolarPVModel
from misc.log_config import configure_logging

configure_logging()
site = Site(latitude=54.60452, longitude=-5.92860, size=100)
pv_panel = SolarPVPanel()
pv_array = SolarPVArray(pv_panel, 3, 35, 83)
pv_model = SolarPVModel(site, pv_array)
pv_model.summary

In [ ]:
import plotly.express as px

px.bar(pv_model.summary_grouped.monthly[["E_Beam_kWm2_Avg", "E_Diffuse_kWm2_Avg", "E_Ground_kWm2_Avg"]])

In [ ]:
px.bar(pv_model.summary_grouped.weekly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total", "Low_Light_Loss_kWh_Total"]])

In [ ]:
# More directed implementation
# %%timeit

import pandas as pd
import plotly.express as px
import seaborn as sns

from meteo.Site import Site
from solar.SolarPVPanel import SolarPVPanel
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging

configure_logging()
pd.options.display.float_format = '{:.3f}'.format

# Set up site location and get TMY data
name = ""
address = "York St, Belfast, BT15 1ED"
client = "Ulster University"

site = Site(name=name, 
            address=address, 
            client=client, 
            latitude=54.60452, 
            longitude=-5.92860, 
            size=100)

In [ ]:
# %%timeit
import time
import logging
start_time = time.time()

# Set up PV panel being used
pv_panel = SolarPVPanel()

# Set up PV arrays
num_pv_panels = 4
surface_pitch = 35
surface_azimuth = -90
albedo = 0.2
cost_per_kWp = 1250

pv_array_1 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, surface_azimuth, albedo, cost_per_kWp)
pv_array_2 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, -45)
pv_array_3 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 0)
pv_array_4 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 45)
pv_array_5 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 90)

pv_arrays = [pv_array_1, pv_array_2, pv_array_3, pv_array_4, pv_array_5]

# Run the model
pv_model = SolarPVModel(site, pv_arrays)

print("")
print(f"{sum(array.pv_kwp for array in pv_model.arrays)}kWp Solar PV modelled")

elapsed_time = time.time() - start_time
logging.info(f"Solar PV model simulations completed in {elapsed_time:.4f} seconds.")


In [ ]:
import plotly.express as px

px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["E_Beam_kWm2_Avg", "E_Diffuse_kWm2_Avg", "E_Ground_kWm2_Avg"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["Array_Temp_C", "Ambient_Temperature_C"]])

In [ ]:
px.line(pv_model.summary_grouped.monthly[["ET_HRad_kWm2_Avg", "E_POA_kWm2_Avg", "Panel_POA_kWm2_Avg"]])

In [ ]:
pv_model.summary

In [ ]:
pv_model.combined_model

# ***Visualisation***

In [ ]:
import plotly.express as px
import seaborn as sns

In [ ]:
print(f"PV Gen: {round(pv_model.all_models.PV_Gen_kWh_Array_1.sum(), 3)}kWh")
print(f"E POA: {round(pv_model.all_models.E_POA_kWm2_Array_1.sum(), 3)}Whm2")

In [ ]:
px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total", "Low_Light_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total", "Low_Light_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.weekly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total", "Low_Light_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.daily[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total", "Low_Light_Loss_kWh_Total"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["Array_Temp_C", "Ambient_Temperature_C"]])


In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["Array_Temp_C", "Ambient_Temperature_C"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

# ***VISUAL TESTING***

In [ ]:
variables = ['E_Beam_kWm2', 'E_Diffuse_kWm2', 'E_Ground_kWm2', 'E_POA_kWm2', 'ET_HRad_kWm2', 
            'Array_Temp_C', 'PV_Gen_kWh', 'AOI', 'Zenith_Angle']

testy = pv_model.all_models

variable = "PV_Gen_kWh"

variable_1 = f"{variable}_Array_1"
variable_2 = f"{variable}_Array_2"
variable_3 = f"{variable}_Array_3"
variable_4 = f"{variable}_Array_4"
variable_7 = f"{variable}_Total"
variable_8 = f"{variable}_Avg"

if variable_1 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_1].mean())
if variable_2 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_2].mean())
if variable_3 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_3].mean())
if variable_4 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_4].mean())
if variable_7 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_7].mean(), alpha=0.4, ls="--")
if variable_8 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_8].mean(), alpha=0.4, ls="--")

In [ ]:
sns.lineplot(pv_model.combined_model["Array_Temp_C_Avg"], alpha = 0.2, ls = "--", color = "orange")
sns.lineplot(pv_model.combined_model["Ambient_Temperature_C"])

In [ ]:
sns.barplot(pv_model.summary_grouped.hourly.Array_Temp_C_Avg)

In [ ]:
sns.lineplot(pv_model.combined_model.groupby("Hour_of_Day")["PV_Gen_kWh_Total"].mean())

In [ ]:
pv_model.all_models["PV_Gen_kWh_Total"].sum()

In [ ]:
pv_model.all_models.columns

In [ ]:
sns.lineplot(pv_model.all_models["PV_Thermal_Loss_kWh_Total"])

In [ ]:
temp_losses = (pv_model.all_models["PV_Thermal_Loss_kWh_Total"].sum() / pv_model.all_models["PV_Gen_kWh_Total"].sum() * 100)

print(f"{round(temp_losses, 3)}% change to power due to temperature losses")

In [ ]:
pv_model.models[0]["model_result"]

In [ ]:
pv_model.all_models

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.daily

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.monthly

# ***EXPERIMENTAL***

# Low Light Losses Visualisation Test

In [ ]:
import numpy as np
import plotly.express as px

# Define the modified logistic function
def calc_low_light_losses(pv_kwp, e_poa, k=0.0075, midpoint=25):
    """
    Modified logistic function to calculate efficiency based on irradiance,
    with a minimum efficiency level.

    Parameters:
    pv_kwp (float): The rated solar PV size (kWp).
    e_poa (float): The irradiance incident on array (W/m2).
    k (float): The steepness of the curve.
    midpoint (float): The irradiance at which the efficiency is at its midpoint.

    Returns:
    float: The calculated efficiency at the given irradiance.
    """
    pv_kwp_min = pv_kwp * 0.6
    eff = pv_kwp_min + (pv_kwp - pv_kwp_min) / (1 + np.exp(-k * (e_poa - midpoint)))
    return eff


# Generate a range of irradiance values from 0 to 1000 W/m2 and set L to 1 kWp
L = 1
irradiance_range = np.linspace(0, 1000, 1001)
efficiencies = calc_low_light_losses(L, irradiance_range)

# Plot Graph of results
fig = px.line(efficiencies[:], y=efficiencies[:], x = irradiance_range[:])

fig.update_layout(xaxis_title="Irradiation (W/m2)", yaxis_title="kWp Output",
                  title = "Line graph showing PV kWp output due to low irradiance losses")

In [ ]:
calc_low_light_losses(1, 200)

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", 
                                         "PV_Thermal_Loss_kWh_Total", "Low_Light_Loss_kWh_Total"]])

In [ ]:
pv_model.summary_grouped.hourly.columns

# Temperature Tests

In [ ]:
import math

def calc_array_temp_homer(
    e_poa,
    ambient_temp,
    cell_temp_coeff=-0.0035,
    electrical_eff=0.21,
    cell_NOCT=42,
    ambient_NOCT=20,
    e_poa_NOCT=800,
    cell_temp_STC=25,
    transmittance_absorptance=0.9,
):
    """Calculates the cell temperature of a PV panel.

    Parameters:
    - e_poa: Plane of array irradiance in kW/m^2.
    - ambient_temp: Ambient temperature in degrees Celsius.
    - cell_temp_coeff: Temperature coefficient of the PV cell.
    - electrical_eff: Electrical efficiency of the PV panel.
    - cell_NOCT, ambient_NOCT: Nominal operating cell temperature and the corresponding ambient temperature.
    - e_poa_NOCT: Irradiance at NOCT conditions in W/m^2.
    - cell_temp_STC: Cell temperature at standard test conditions in degrees Celsius.
    - transmittance_absorptance: Transmittance and absorptance product of the PV panel.

    Returns:
    - Cell temperature of the PV panel.
    """
    temp_factor = (cell_NOCT - ambient_NOCT) * ((e_poa * 1000) / e_poa_NOCT)
    numerator = ambient_temp + temp_factor * (
        1
        - (electrical_eff * (1 - cell_temp_coeff * cell_temp_STC))
        / transmittance_absorptance
    )
    denominator = 1 + temp_factor * (
        cell_temp_coeff * electrical_eff / transmittance_absorptance
    )

    return numerator / denominator


def calc_array_temp_pvsyst(e_poa, ambient_temp, windspeed, uc=29, uv=1.2):
    u = uc + (uv * windspeed)
    tcell = ambient_temp + (1/u) * (0.9 * e_poa * 1000 * (1-0.21))
    return tcell

def calc_array_temp_sandia(e_poa: float, ambient_temp: float, wind_speed: float, 
                           a: float = -3.47, b: float = -0.0594) -> float:
    """
    Calculate the temperature of a photovoltaic (PV) array based on the Sandia method.
    
    Parameters:
    - e_poa (float): Plane of array irradiance in kW/m^2. Represents the solar irradiance incident on the PV array.
    - ambient_temp (float): Ambient temperature in degrees Celsius.
    - wind_speed (float): Wind speed in m/s at the site of the PV array.
    - a (float): Coefficient a in the exponential model, defaulting to -3.47.
    - b (float): Coefficient b in the exponential model, defaulting to -0.0594.
    
    Returns:
    - float: Estimated temperature of the PV array in degrees Celsius.
    """
    array_temp = e_poa * 1000 * math.exp(a + b * wind_speed) + ambient_temp
    return array_temp

def calc_array_temp_faiman(e_poa, ambient_temp, wind_speed, U_0=25, U_1=6.84):
    array_temp = ambient_temp + (e_poa * 1000 / (U_0 + U_1 * wind_speed))
    return array_temp


# Example usage:
e_poa=0.5
ambient_temp=25
wind_speed=1

list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []


print("")
wind_speed=10
for i in range(1):
    cell_temperature = calc_array_temp_homer(e_poa, ambient_temp)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Homer model with {wind_speed}m/s windspeed")
    list_2.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(26):
    cell_temperature = calc_array_temp_pvsyst(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the PVSyst model with {wind_speed}m/s windspeed")
    list_3.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(26):
    cell_temperature = calc_array_temp_sandia(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Sandia model with {wind_speed}m/s windspeed")
    list_4.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(26):
    cell_temperature = calc_array_temp_faiman(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Faiman model with {wind_speed}m/s windspeed")
    list_5.append(cell_temperature)
    wind_speed += 1

In [ ]:
%%timeit
cell_temperature = calc_array_temp_homer(e_poa, ambient_temp)

In [ ]:
%%timeit
cell_temperature = calc_array_temp_faiman(e_poa, ambient_temp, wind_speed)

In [ ]:
%%timeit
cell_temperature = calc_array_temp_sandia(e_poa, ambient_temp, wind_speed)

In [ ]:
%%timeit
cell_temperature = calc_array_temp_pvsyst(e_poa, ambient_temp, wind_speed)